https://assaeunji.github.io/machine%20learning/2020-11-29-implicitfeedback/

# 전체 Orders 모델 학습
- 한의약 도서이기 때문에 시즌성이 반영이 안되어도 괜찮다고 판단
- 전체 2년치 orders 데이터로 MF,LMF,MP 학습 및 평가 진행

# 추천 모델
- ALS MF, LMF, MP (총 3개)
- 총 3개의 추천을 진행하며 MF와 LMF 의 경우 콜드스타트 유저(신규 유저)인 경우 MP로 추천 진행

In [223]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

import scipy.sparse as sparse
import random
import implicit
from implicit.als import AlternatingLeastSquares as ALS

%cd /home/user_3/medistream-recsys/Script
from preprocessing import drop_columns,dict_to_column,dict_to_set,set_to_column,key_to_element

pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 100)

/home/user_3/medistream-recsys/Script


# 1.Dataload

In [224]:
# products name 확인 용
products_df = pd.read_json("/fastcampus-data/products/products.json")
products_df = key_to_element(['_id'],products_df)

100%|██████████| 5141/5141 [00:00<00:00, 706008.67it/s]


In [225]:
df = pd.read_json('/fastcampus-data/select_column_version_4.json')

In [226]:
df['date_paid'] = pd.to_datetime(df['date_paid'])

In [227]:
# 전체 기간
df['date_paid'].min(), df['date_paid'].max()

(Timestamp('2019-08-26 02:41:49.950000+0000', tz='UTC'),
 Timestamp('2022-09-13 08:59:21.151000+0000', tz='UTC'))

In [228]:
# paid orders만 가져오기
df['date_paid'] = pd.to_datetime(df['date_paid'])
df_only_paid = df[~df['date_paid'].isna()]
# 취소 안된 것만 가져오기
complete_df = df_only_paid[(df_only_paid['paid'] == True) & (df_only_paid['cancelled']==False)]
# 도서 카테고리만 가져오기
only_book = complete_df[complete_df['name'] == '도서']

# 유저가 중복으로 아이템 구매 삭제
df_duplicated_book = only_book.drop_duplicates(subset=['customer_id','product_ids'])
df_book = df_duplicated_book.sort_values(by='date_paid').reset_index(drop=True)

# 도서, 소모품 카테고리
# df_book = complete_df[complete_df['name'].isin(['도서','소모품'])].sort_values(by='date_paid')

In [229]:
# none 값 확인하기
df_book.isna().sum()

_id                 0
date_created        0
regular_price       0
sale_price          0
three_months        0
date_paid           0
customer_id         0
paid                0
name_x              0
category_id_y       0
product_ids         0
quantity            0
price               0
price_total         0
age_group        3007
한의사 여부             79
사업자 여부             79
cancelled           0
name                0
slug                0
dtype: int64

## 전체 데이터 EDA

In [230]:
print('중복 제거 전:',len(only_book), '중복 제거 후:',len(df_book))

중복 제거 전: 38395 중복 제거 후: 37866


In [231]:
print('전체 데이터 수:',len(df_book))

전체 데이터 수: 37866


In [232]:
print('아이템 수:',len(df_book.product_ids.unique()),'유저 수:',len(df_book.customer_id.unique()))

아이템 수: 342 유저 수: 7410


## 전체 아이템 중복 확인

In [233]:
# product_ids, name_x 일치하지 않음, 전처리 필요
len(df_book.product_ids.unique()), len(df_book.name_x.unique())

(342, 370)

In [234]:
# 중복 제거 후 수 비교 확인
len(df_book.drop_duplicates(subset=['product_ids','name_x']).name_x.unique())

370

In [235]:
product_name_preprocess_df = df_book.copy()

In [236]:
# 각 마지막 product_ids, name으로 채우기
product_ids_to_name = {}
for idx, row in product_name_preprocess_df.iterrows():
    product_ids_to_name[row.product_ids] = row.name_x
product_name_preprocess_df['name_x'] = product_name_preprocess_df['product_ids'].apply(lambda x: product_ids_to_name[x])

name_to_product_ids = {}
for idx, row in product_name_preprocess_df.iterrows():
    name_to_product_ids[row.name_x] = row.product_ids
product_name_preprocess_df['product_ids'] = product_name_preprocess_df['name_x'].apply(lambda x: name_to_product_ids[x])

In [237]:
# product_ids, name_x 일치 확인
len(product_name_preprocess_df.product_ids.unique()), len(product_name_preprocess_df.name_x.unique())

(340, 340)

# 2.train test split
- 전체 37개월 중 마지막 6개월치 데이터를 test 선정
- train 없는 test 아이템을 삭제 진행합니다.

In [238]:
product_name_preprocess_df['date_paid'].max()

Timestamp('2022-09-13 08:51:40+0000', tz='UTC')

In [239]:
product_name_preprocess_df['date_paid'].min()

Timestamp('2019-09-23 00:25:41.241000+0000', tz='UTC')

In [240]:
from datetime import datetime, timedelta

# 6개월 test (9-6 = 3)
date = '2022-03-13'

train_before_preprocess = product_name_preprocess_df[product_name_preprocess_df['date_paid'] < date]
test_before_preprocess = product_name_preprocess_df[product_name_preprocess_df['date_paid'] >= date]

## train test 아이템 중복 확인

In [241]:
len(train_before_preprocess.product_ids.unique()),len(test_before_preprocess.product_ids.unique())

(277, 275)

In [242]:
len(set(train_before_preprocess.product_ids.unique())-set(test_before_preprocess.product_ids.unique()))

65

In [243]:
# test 아이템에 train 없는 아이템 확인
len(set(test_before_preprocess.product_ids.unique())-set(train_before_preprocess.product_ids.unique()))

63

In [244]:
# test 만 있는 item 제거
only_test_items = set(test_before_preprocess.product_ids.unique())-set(train_before_preprocess.product_ids.unique())
test = test_before_preprocess[~test_before_preprocess['product_ids'].isin(only_test_items)]

In [245]:
len(test.customer_id.unique()), len(train_before_preprocess.customer_id.unique())

(1948, 6605)

In [246]:
len(test.customer_id.unique())

1948

In [247]:
# train 변수 명 변경
train = train_before_preprocess.copy()

# train test eda

### 전처리 전후 비교

In [248]:
print('train 전처리 전:',len(train_before_preprocess), 'train 전처리 후:',len(train))

train 전처리 전: 29576 train 전처리 후: 29576


In [249]:
print('test 전처리 전:',len(test_before_preprocess), 'test 전처리 후:',len(test))

test 전처리 전: 8290 test 전처리 후: 3787


- test를 마지막 6개월로 잡을 경우 5000 건 가량 절반 이상 줄어들어서 유의미한 최신 경향을 좀더 반영해야 됨

In [250]:
print('test 전처리 전 아이템:',len(set(test_before_preprocess.product_ids)), 'test 전처리 후 아이템:',len(set(test.product_ids)))

test 전처리 전 아이템: 275 test 전처리 후 아이템: 212


### user 수 비교 

In [251]:
print('train 유저 수:',len(train.customer_id.unique()))

train 유저 수: 6605


In [252]:
print('test 유저 수:',len(test.customer_id.unique()))

test 유저 수: 1948


In [253]:
# 신규 유저는 MP 같은 다른 방법으로 추천 진행해야 함
print('test 만 있는 신규 유저 :',len(set(test['customer_id'].unique())- set(train['customer_id'].unique())))

test 만 있는 신규 유저 : 484


### item 개수 비교

In [254]:
print('train 아이템 수 :',len(set(train.product_ids)), 'test 아이템 수 :',len(set(test.product_ids)))

train 아이템 수 : 277 test 아이템 수 : 212


In [255]:
print('train 만 있는 아이템 수:',  len(set(train.product_ids)-set(test.product_ids)))

train 만 있는 아이템 수: 65


In [256]:
print('test 만 있는 아이템 수:', len(set(test.product_ids) - set(train.product_ids)))

test 만 있는 아이템 수: 0


# 3. sparse matrix 만들기

## ALS MF Matrix

In [257]:
PdIds = train.product_ids.unique()

PdIdToIndex = {}
indexToPdId = {}

colIdx = 0

for PdId in PdIds:
    PdIdToIndex[PdId] = colIdx
    indexToPdId[colIdx] = PdId
    colIdx += 1
    
userIds = train.customer_id.unique()

userIdToIndex = {}
indexToUserId = {}

rowIdx = 0

for userId in userIds:
    userIdToIndex[userId] = rowIdx
    indexToUserId[rowIdx] = userId
    rowIdx += 1

import scipy.sparse as sp

rows = []
cols = []
vals = []

for row in train.itertuples():
    rows.append(userIdToIndex[row.customer_id])
    cols.append(PdIdToIndex[row.product_ids])
    vals.append(1)

purchase_sparse = sp.csr_matrix((vals, (rows, cols)), shape=(rowIdx,colIdx))

matrix = purchase_sparse.todense()
matrix

matrix([[1, 0, 0, ..., 0, 0, 0],
        [0, 1, 0, ..., 0, 0, 0],
        [0, 1, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

### Most_popular_matrix

In [258]:
most_popular = train.groupby(['product_ids','name_x']).count()['customer_id'].reset_index()
most_popular

,product_ids,name_x,customer_id
0,5d13115e32026c0b35383897,KCD 한방내과 진찰진단 가이드라인,704
1,5d132e964b25b80d9fb1f352,노인요양병원 진료지침서,63
2,5d5373e94e77525ec5ca1135,통증치료를 위한 근육 초음파와 주사 테크닉,238
3,5d59ee854e77525ec5ca1212,일차진료 한의사를 위한 보험한약입문 - 둘째 판,433
4,5d77b31f19efa30eb29143c9,NEO 인턴 핸드북,295
5,5d77b73619efa30eb29143ca,병태생리 Visual map,121
6,5d78491b19efa30eb29143cc,플로차트 한약치료,223
7,5d784bac19efa30eb29143d0,Medical acupuncture '침의 과학적 접근과 임상활용',483
8,5d784e1719efa30eb29143d1,"근골격계 질환의 진단 및 재활치료, 4판",143
9,5d78505019efa30eb29143d2,개원의를 위한 통증사냥법,281


### Medistream_prediction_matrix
- 메디스트림 메디마켓에서 제공하는 정렬 추천 성능 비교를 위한 df 구현
- 인기도순, 최신순, 과거순, 높은 가격순, 낮은 가격순, 이름순 (총 6 가지)
- 각각 구현해보고 학습 모델 대비 성능 비교

In [259]:
medistream_prediction_df = train[['date_created','regular_price','sale_price','three_months','product_ids','name_x']]
medistream_prediction_preprop_df = medistream_prediction_df.drop_duplicates(subset=['product_ids'], ignore_index=True)
medistream_prediction_preprop_df['date_created'] = pd.to_datetime(medistream_prediction_preprop_df['date_created'])
# sale_prices가 0이면 regular_price 값으로 채워넣어야하는데 0이 없음(전처리 필요 무)

<ipython-input-259-4d4e36d189e6>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  medistream_prediction_preprop_df['date_created'] = pd.to_datetime(medistream_prediction_preprop_df['date_created'])


# Sparsity 확인

In [260]:
# Sparsity: 얼마나 비어있나?
matrix_size = purchase_sparse.shape[0]* purchase_sparse.shape[1]
num_purchases = len(purchase_sparse.nonzero()[0])
sparsity = 100 * (1 - (num_purchases / matrix_size))
sparsity

98.38362251548848

# 4. Model

# Model 학습 진행
- real test 만들기
- implict 라이브러리 사용(MF,LMF)
- MF 구현 모델 사용

In [261]:
# real test 
ground_trues = []
for user_id in test['customer_id'].unique():
    ground_trues.append({'id': user_id,\
    'items':list(test[test['customer_id']==user_id].product_ids)
    })

## ALS fit

In [262]:
als_model = ALS(factors=20, regularization=0.01, iterations = 50, random_state=42)
als_model.fit(purchase_sparse)

  0%|          | 0/50 [00:00<?, ?it/s]

In [263]:
# item, user vector 추출
als_item_factors = als_model.item_factors
als_user_factors = als_model.user_factors

In [264]:
# 각 shape 확인
als_item_factors.shape, als_user_factors.shape

((277, 20), (6605, 20))

## LMF fit

In [265]:
from implicit.lmf import LogisticMatrixFactorization as LMF

In [266]:
lmf_model = LMF(factors=20, regularization=0.001, iterations = 20, random_state=42)
lmf_model.fit(purchase_sparse)

  0%|          | 0/20 [00:00<?, ?it/s]

In [267]:
lmf_item_factors = lmf_model.item_factors
lmf_user_factors = lmf_model.user_factors

# 5. prediction

# ALS mf prediction

In [268]:
# 신규 유저인 경우 mp로 넣기
# 전체 도서에 대한 판매 만큼 정렬 후 넣기
most_popular_list = most_popular.sort_values(by='customer_id',ascending=False).index

# test 예측값, 이미 구매 했을 경우 제외
als_predict_list = []
for user_id in test['customer_id'].unique():
    try:
        result = als_model.recommend(userIdToIndex[user_id], purchase_sparse[userIdToIndex[user_id]], N=100)
        als_predict_list.append({'id':user_id ,'items':[indexToPdId[num] for num in result[0]]})
    except:
        train_purchase_list = list(train[train['customer_id']==user_id].product_ids)
        als_predict_list.append({'id':user_id ,'items':[most_popular.product_ids.loc[num] for num in most_popular_list \
                                                            if most_popular.product_ids.loc[num] not in train_purchase_list \
                                                            ]})
        
# 100 개만 예측하기
for idx, pred_list in enumerate(als_predict_list):
    als_predict_list[idx]['items'] = pred_list['items'][:100]

# LMF prediction

In [269]:
# 신규 유저 mp로 넣기
most_popular_list = most_popular.sort_values(by='customer_id',ascending=False).index

# test 예측값
lmf_predict_list = []
for user_id in test['customer_id'].unique():
    try:
        result = lmf_model.recommend(userIdToIndex[user_id], purchase_sparse[userIdToIndex[user_id]], N=100)
        lmf_predict_list.append({'id':user_id ,'items':[indexToPdId[num] for num in result[0]]})
    except:
        train_purchase_list = list(train[train['customer_id']==user_id].product_ids)
        lmf_predict_list.append({'id':user_id ,'items':[most_popular.product_ids.loc[num] for num in most_popular_list \
                                                            if most_popular.product_ids.loc[num] not in train_purchase_list \
                                                            ]})
        
# 100 개만 예측하기
for idx, pred_list in enumerate(lmf_predict_list):
    lmf_predict_list[idx]['items'] = pred_list['items'][:100]

# most popular prediction

In [270]:
# 전체 도서에 대한 판매 만큼 정렬 후 넣기
most_popular_list = most_popular.sort_values(by='customer_id',ascending=False).index

# test 예측값, 이미 구매 했을 경우 제외
predict_popular_list = []
for user_id in test['customer_id'].unique():
    train_purchase_list = list(train[train['customer_id']==user_id].product_ids)
    predict_popular_list.append({'id':user_id ,'items':[most_popular.product_ids.loc[num] for num in most_popular_list \
                                                            if most_popular.product_ids.loc[num] not in train_purchase_list \
                                                            ]})

# 100 개만 예측하기
for idx, pred_list in enumerate(predict_popular_list):
    predict_popular_list[idx]['items'] = pred_list['items'][:100]

# medistream prediction
- 메디스트림 메디마켓에서 제공하는 정렬 추천 성능 비교
- 인기도순, 최신순, 과거순, 높은 가격순, 낮은 가격순, 이름순 (총 6 가지)
- 각각 구현해보고 학습 모델 대비 성능 비교

In [271]:
# 인기도순
medistream_popular_list = medistream_prediction_preprop_df.sort_values(by='three_months', ascending=False).index
# 최신순
medistream_latest_list = medistream_prediction_preprop_df.sort_values(by='date_created', ascending=False).index
# 오랜된 순
medistream_oldest_list = medistream_prediction_preprop_df.sort_values(by='date_created', ascending=True).index
# 높은 가격 순
medistream_high_price_list = medistream_prediction_preprop_df.sort_values(by='sale_price', ascending=False).index
# 낮은 가격 순
medistream_low_price_list = medistream_prediction_preprop_df.sort_values(by='sale_price', ascending=True).index
# 이름 순
medistream_name_sort_list = medistream_prediction_preprop_df.sort_values(by='name_x',ascending=True).index

def medistream_prediction_method(predict_num:int ,medi_predict_list:list)->list:
    medistream_predict_list = []
    for user_id in test['customer_id'].unique():
        medistream_predict_list.append({'id':user_id ,'items':[medistream_prediction_preprop_df.product_ids.loc[num] \
                                                                       for num in medi_predict_list]})

    # 100 개만 예측하기
    for idx, pred_list in enumerate(medistream_predict_list):
        medistream_predict_list[idx]['items'] = pred_list['items'][:predict_num]
        
    return medistream_predict_list

In [272]:
medistream_predict_popular_list = medistream_prediction_method(100, medistream_popular_list)
medistream_predict_latest_list = medistream_prediction_method(100, medistream_latest_list)
medistream_predict_oldest_list = medistream_prediction_method(100, medistream_oldest_list)
medistream_predict_high_price_list = medistream_prediction_method(100, medistream_high_price_list)
medistream_predict_low_price_list = medistream_prediction_method(100, medistream_low_price_list)
medistream_predict_name_sort_list = medistream_prediction_method(100, medistream_name_sort_list)

# 6. evaluation

## NDCG 평가지표

In [273]:
class CustomEvaluator:
    # relavence 모두 1로 동일하게 봄
    def _idcg(self, l):
        return sum((1.0 / np.log(i + 2) for i in range(l)))
    

    def __init__(self):
        self._idcgs = [self._idcg(i) for i in range(1000)]
    '''
    idcgs 예시, item 3개 추천되므로 3.074281787960283 가 됩니다.
    [0, 1.4426950408889634, 2.352934267515801, 3.074281787960283]
    '''

    def _ndcg(self, gt, rec):
        dcg = 0.0
        for i, r in enumerate(rec):
            if r in gt:
                dcg += 1.0 / np.log(i + 2)

        return dcg / self._idcgs[len(gt)]

    def _eval(self, gt_list, rec_list):
        gt_dict = {g["id"]: g for g in gt_list}
        ndcg_score = 0.0

        for rec in rec_list:
            gt = gt_dict[rec["id"]]
            ndcg_score += self._ndcg(gt["items"], rec["items"])


        ndcg_score = ndcg_score / len(rec_list)


        return ndcg_score

    def evaluate(self, gt_list, rec_list):
        try:
            ndcg_score = self._eval(gt_list, rec_list)
            print(f"nDCG: {ndcg_score:.6}")
        except Exception as e:
            print(e)


# ALS NDCG

In [274]:
# ALS 
evaluator = CustomEvaluator()
evaluator.evaluate(ground_trues, als_predict_list)

nDCG: 0.241069


In [275]:
len(als_predict_list),len(ground_trues)

(1948, 1948)

In [276]:
# 아이템 맞춘 개수
cnt = 0
for gt, pred_list in zip(ground_trues, als_predict_list):
    for pred in pred_list['items']:
        if pred in gt['items']:
            cnt += 1
cnt

2297

# LMF NDCG

In [277]:
# ALS 
evaluator = CustomEvaluator()
evaluator.evaluate(ground_trues, lmf_predict_list)

nDCG: 0.253031


In [278]:
len(lmf_predict_list),len(ground_trues)

(1948, 1948)

In [279]:
# 아이템 맞춘 개수
cnt = 0
for gt, pred_list in zip(ground_trues, lmf_predict_list):
    for pred in pred_list['items']:
        if pred in gt['items']:
            cnt += 1
cnt

2503

# most popular NDCG

In [280]:
# most popular
evaluator = CustomEvaluator()
evaluator.evaluate(ground_trues, predict_popular_list)

nDCG: 0.363957


In [281]:
len(predict_popular_list),len(ground_trues)

(1948, 1948)

In [282]:
# 아이템 맞춘 개수
cnt = 0
for gt, pred_list in zip(ground_trues, predict_popular_list):
    for pred in pred_list['items']:
        if pred in gt['items']:
            cnt += 1
cnt

2812

## medistream prediction NDCG

In [283]:
def medistream_prediction(ground_trues:list, predict_list:list):
    evaluator = CustomEvaluator()
    ndcg = evaluator._eval(ground_trues, predict_list)
    
    assert len(predict_list) == len(ground_trues)
    
    cnt = 0
    for gt, pred_list in zip(ground_trues, predict_list):
        for pred in pred_list['items']:
            if pred in gt['items']:
                cnt += 1
    return ndcg, cnt

In [284]:
medistream_predict_score = {'medistream_predict':['medi_popular','latest','oldest','higt_price','low_price','name_sort'], \
                            'ndcg':[], 'cnt':[]}

medistream_predict_list = [medistream_predict_popular_list, medistream_predict_latest_list, medistream_predict_oldest_list,\
                          medistream_predict_high_price_list, medistream_predict_low_price_list, medistream_predict_name_sort_list]

for medistream_predict in medistream_predict_list:
    ndcg, cnt = medistream_prediction(ground_trues, medistream_predict)
    medistream_predict_score['ndcg'].append(ndcg)
    medistream_predict_score['cnt'].append(cnt)
pd.DataFrame(medistream_predict_score)    

,medistream_predict,ndcg,cnt
0,medi_popular,0.455699,3428
1,latest,0.157675,2059
2,oldest,0.120806,1390
3,higt_price,0.090534,1599
4,low_price,0.094467,1486
5,name_sort,0.066372,1032


# 7. hyper parameter tuning

## 7-1. ALS MF hypter parameter tuning

In [285]:
als_mf_hyper_parameter = {'factor':[],'regularization':[],'iteration':[],'NDCG':[]}

factors = [5,10,15,20]
regularizations = [0.01,0.005]
iterations = [5,10,15,20,25,30,40,50]

for factor in factors:
    for regularization in regularizations:
        for iteration in iterations:
            als_model = ALS(factors=factor, regularization=regularization, iterations = iteration, random_state=42)
            als_model.fit(purchase_sparse, show_progress=False)
            
            # 신규 유저인 경우 mp로 넣기
            # 전체 도서에 대한 판매 만큼 정렬 후 넣기
            most_popular_list = most_popular.sort_values(by='customer_id',ascending=False).index

            # test 예측값, 이미 구매 했을 경우 제외
            als_predict_list = []
            for user_id in test['customer_id'].unique():
                try:
                    result = als_model.recommend(userIdToIndex[user_id], purchase_sparse[userIdToIndex[user_id]], N=100)
                    als_predict_list.append({'id':user_id ,'items':[indexToPdId[num] for num in result[0]]})
                except:
                    train_purchase_list = list(train[train['customer_id']==user_id].product_ids)
                    als_predict_list.append({'id':user_id ,'items':[most_popular.product_ids.loc[num] for num in most_popular_list \
                                                                        if most_popular.product_ids.loc[num] not in train_purchase_list \
                                                                        ]})

            # 100 개만 예측하기
            for idx, pred_list in enumerate(als_predict_list):
                als_predict_list[idx]['items'] = pred_list['items'][:100]
                
            # ALS 
            evaluator = CustomEvaluator()
            ndcg = evaluator._eval(ground_trues, als_predict_list)
            
            als_mf_hyper_parameter['factor'].append(factor)
            als_mf_hyper_parameter['regularization'].append(regularization)
            als_mf_hyper_parameter['iteration'].append(iteration)
            als_mf_hyper_parameter['NDCG'].append(ndcg)

In [286]:
pd.DataFrame(als_mf_hyper_parameter).sort_values(by='NDCG',ascending=False).head()

,factor,regularization,iteration,NDCG
9,5,0.005,10,0.290966
1,5,0.010,10,0.290811
2,5,0.010,15,0.290551
10,5,0.005,15,0.290507
0,5,0.010,5,0.289504


## 7-2. LMF hypter parameter tuning

In [287]:
lmf_hyper_parameter = {'factor':[],'regularization':[],'iteration':[],'NDCG':[]}

factors = [5,10,15,20]
regularizations = [0.01,0.005]
iterations = [5,10,15,20,25,30,40,50]

for factor in factors:
    for regularization in regularizations:
        for iteration in iterations:
            lmf_model = LMF(factors=factor, regularization=regularization, iterations = iteration, random_state=42)
            lmf_model.fit(purchase_sparse, show_progress=False)
            
            # 신규 유저 mp로 넣기
            most_popular_list = most_popular.sort_values(by='customer_id',ascending=False).index

            # test 예측값
            lmf_predict_list = []
            for user_id in test['customer_id'].unique():
                try:
                    result = lmf_model.recommend(userIdToIndex[user_id], purchase_sparse[userIdToIndex[user_id]], N=100)
                    lmf_predict_list.append({'id':user_id ,'items':[indexToPdId[num] for num in result[0]]})
                except:
                    train_purchase_list = list(train[train['customer_id']==user_id].product_ids)
                    lmf_predict_list.append({'id':user_id ,'items':[most_popular.product_ids.loc[num] for num in most_popular_list \
                                                                        if most_popular.product_ids.loc[num] not in train_purchase_list \
                                                                        ]})

            # 100 개만 예측하기
            for idx, pred_list in enumerate(lmf_predict_list):
                lmf_predict_list[idx]['items'] = pred_list['items'][:100]
                
            # LMF
            evaluator = CustomEvaluator()
            ndcg = evaluator._eval(ground_trues, lmf_predict_list)
            
            lmf_hyper_parameter['factor'].append(factor)
            lmf_hyper_parameter['regularization'].append(regularization)
            lmf_hyper_parameter['iteration'].append(iteration)
            lmf_hyper_parameter['NDCG'].append(ndcg)

In [288]:
pd.DataFrame(lmf_hyper_parameter).sort_values(by='NDCG',ascending=False).head()

,factor,regularization,iteration,NDCG
63,20,0.005,50,0.258198
55,20,0.010,50,0.257807
54,20,0.010,40,0.253805
62,20,0.005,40,0.253636
59,20,0.005,20,0.253291


# 8. 결론

- test 전처리 전: 8290 test 전처리 후: 3787
- 5000 건 가량 전처리되었기 때문에 옳지 않은 실험인 것 같음
- als mf : 0.290966 (factor: 5, regularization: 0.005, iteration: 10)
- lmf : 0.258198 (factor: 20, regularization: 0.005, iteration: 50)
- mp : 0.363957

- mp 가 압도적으로 높은 성능을 보임

# 9. 추천된 items 확인

In [289]:
products_df = pd.read_json("/fastcampus-data/products/products.json")
products_df = key_to_element(['_id'],products_df)

100%|██████████| 5141/5141 [00:00<00:00, 685276.71it/s]


In [290]:
# pred_item, rea_item 비교
def pred_real_dataframe(user_num):
    pred_items_names = []
    predict_dict = als_predict_list[user_num]['items']
    for item in predict_dict:
        pred_items_names.append(products_df[products_df['_id'] == item].meta_title.unique())

    real_items_names = []
    trues_dict = ground_trues[user_num]['items']
    for item in trues_dict:
        real_items_names.append(products_df[products_df['_id'] == item].meta_title.unique())
    return pd.DataFrame({'pred_item':pred_items_names,'real_item':real_items_names})
    

In [291]:
# train_item, pred_item, real_item 비교
def train_pred_items(user_nums):
    train_pred_items_df = pd.DataFrame(columns=['train_item','pred_item'])
    for user_num in range(1,user_nums):
        train_item_names = []
        for idx in grouped_purchased[grouped_purchased['customer_id']==ground_trues[user_num]['id']].product_ids:
            train_item_names.append(products_df[products_df['_id'] == idx].meta_title.unique()[0])

        pred_items_names = []
        predict_dict = als_predict_list[user_num]['items']
        for item in predict_dict:
            pred_items_names.append(products_df[products_df['_id'] == item].meta_title.unique())


        
        train_pred_items_df.loc[user_num,'train_item'] = train_item_names
        train_pred_items_df.loc[user_num,'pred_item'] = pred_items_names
    return train_pred_items_df


In [292]:
train_pred_items(13)

NameError: name 'grouped_purchased' is not defined

In [ ]:
# 예측 유저 구매 횟수 확인
pd.DataFrame(purchase_sparse[1].todense()).T.value_counts()

In [294]:
print('첫날 :',train['date_paid'].min(),'마지막 날:',train['date_paid'].max())
print('train 총 기간:',train['date_paid'].max()-train['date_paid'].min())
print('______________________________________________________')
print('첫날 :',test['date_paid'].min(),'마지막 날:',test['date_paid'].max())
print('test 총 기간:',test['date_paid'].max()-test['date_paid'].min())
print('______________________________________________________')
print('train 데이터수:', len(train))
print('______________________________________________________')
print('train 유저수:',len(set(train.customer_id)))
print('______________________________________________________')
print('test 데이터수:',len(test))
print('______________________________________________________')
print('test 유저수:',len(set(test.customer_id)))
print('______________________________________________________')
print('test 신규 유저 수:',len(set(test['customer_id'].unique())- set(train['customer_id'].unique())))
print('______________________________________________________')
print('test 신규 아이템 수:',len(set(test_before_preprocess.product_ids.unique())-set(train_before_preprocess.product_ids.unique())))
# test 전처리 진행했을 경우
print('______________________________________________________')
print('원본 test 수:', len(test))
print('______________________________________________________')
# print('전처리 진행했을 경우 test 수:', len(if_prepro_test))
print('______________________________________________________')
print('mf')
display(pd.DataFrame(als_mf_hyper_parameter).sort_values(by='NDCG',ascending=False).head(2))
print('______________________________________________________')
print('lmf')
display(pd.DataFrame(lmf_hyper_parameter).sort_values(by='NDCG',ascending=False).head(2))
print('______________________________________________________')
print('mp')
evaluator.evaluate(ground_trues, predict_popular_list)
print('______________________________________________________')
display(pd.DataFrame(medistream_predict_score))

첫날 : 2019-09-23 00:25:41.241000+00:00 마지막 날: 2022-03-12 23:51:57.779000+00:00
train 총 기간: 901 days 23:26:16.538000
______________________________________________________
첫날 : 2022-03-13 00:39:22.961000+00:00 마지막 날: 2022-09-13 08:51:40+00:00
test 총 기간: 184 days 08:12:17.039000
______________________________________________________
train 데이터수: 29576
______________________________________________________
train 유저수: 6605
______________________________________________________
test 데이터수: 3787
______________________________________________________
test 유저수: 1948
______________________________________________________
test 신규 유저 수: 484
______________________________________________________
test 신규 아이템 수: 63
______________________________________________________
원본 test 수: 3787
______________________________________________________
______________________________________________________
mf


,factor,regularization,iteration,NDCG
9,5,0.005,10,0.290966
1,5,0.010,10,0.290811


______________________________________________________
lmf


,factor,regularization,iteration,NDCG
63,20,0.005,50,0.258198
55,20,0.010,50,0.257807


______________________________________________________
mp
nDCG: 0.363957
______________________________________________________


,medistream_predict,ndcg,cnt
0,medi_popular,0.455699,3428
1,latest,0.157675,2059
2,oldest,0.120806,1390
3,higt_price,0.090534,1599
4,low_price,0.094467,1486
5,name_sort,0.066372,1032
